In [1]:
# BASE
# ------------------------------------------------------
import numpy as np
import pandas as pd
import os
import gc

# PACF - ACF
# ------------------------------------------------------
import statsmodels.api as sm

# DATA VISUALIZATION
# ------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


# CONFIGURATIONS
# ------------------------------------------------------
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# 데이터 불러오기
import os
address_ = r'D:\Store_Sales/'
df_names = ['holidays', 'oil', 'sample_submission', 'stores', 'test', 'train', 'transactions']
for i,files in enumerate(os.listdir(path=address_)): 
    globals()[df_names[i]] = pd.read_csv(address_+files, encoding='cp949')

In [3]:
holidays = holidays
oil = oil
sample_submission = sample_submission
stores = stores
train = train
transactions = transactions.sort_values(["store_nbr", "date"])

In [5]:
# 데이터 타입 변경
# Datetime
train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)
transactions["date"] = pd.to_datetime(transactions.date)

# Data types
train.onpromotion = train.onpromotion.astype(float)
train.sales = train.sales.astype(float)
stores.cluster = stores.cluster.astype(int)

In [11]:
# Oil 데이터 interpolation을 통해 NA값 채우기
 oil["date"] = pd.to_datetime(oil.date)
# Resample
oil = oil.set_index("date").dcoilwtico.resample("D").sum().reset_index()
# Interpolate
oil["dcoilwtico"] = np.where(oil["dcoilwtico"] == 0, np.nan, oil["dcoilwtico"])
oil["dcoilwtico_interpolated"] =oil.dcoilwtico.interpolate()
# Plot
p = oil.melt(id_vars=['date']+list(oil.keys()[5:]), var_name='Legend')
px.line(p.sort_values(["Legend", "date"], ascending = [False, True]), x='date', y='value', color='Legend',title = "Daily Oil Price" )

In [13]:
# Sales가 전혀 존재하지 않는 row, 즉 가게가 open하기 전인 것으로 보여지는 row 제거
train = train[~((train.store_nbr == 52) & (train.date < "2017-04-20"))]
train = train[~((train.store_nbr == 22) & (train.date < "2015-10-09"))]
train = train[~((train.store_nbr == 42) & (train.date < "2015-08-21"))]
train = train[~((train.store_nbr == 21) & (train.date < "2015-07-24"))]
train = train[~((train.store_nbr == 29) & (train.date < "2015-03-20"))]
train = train[~((train.store_nbr == 20) & (train.date < "2015-02-13"))]
train = train[~((train.store_nbr == 53) & (train.date < "2014-05-29"))]
train = train[~((train.store_nbr == 36) & (train.date < "2013-05-09"))]
train.shape

(3000888, 6)


(2780316, 6)

In [15]:
# store_nbr에 따라 전혀 매출이 존재하지 않는 제품군 제거 
c = train.groupby(["store_nbr", "family"]).sales.sum().reset_index().sort_values(["family","store_nbr"])
c = c[c.sales == 0]
outer_join = train.merge(c[c.sales == 0].drop("sales",axis = 1), how = 'outer', indicator = True)
train = outer_join[~(outer_join._merge == 'both')].drop('_merge', axis = 1)
del outer_join
gc.collect()

(2780316, 6)


(2698648, 6)

In [16]:
zero_prediction = []
for i in range(0,len(c)):
    zero_prediction.append(
        pd.DataFrame({
            "date":pd.date_range("2017-08-16", "2017-08-31").tolist(),
            "store_nbr":c.store_nbr.iloc[i],
            "family":c.family.iloc[i],
            "sales":0
        })
    )
zero_prediction = pd.concat(zero_prediction)
del c
gc.collect()
zero_prediction

,date,store_nbr,family,sales
0,2017-08-16,1,BABY CARE,0
1,2017-08-17,1,BABY CARE,0
2,2017-08-18,1,BABY CARE,0
3,2017-08-19,1,BABY CARE,0
4,2017-08-20,1,BABY CARE,0
...,...,...,...,...
11,2017-08-27,54,LAWN AND GARDEN,0
12,2017-08-28,54,LAWN AND GARDEN,0
13,2017-08-29,54,LAWN AND GARDEN,0
14,2017-08-30,54,LAWN AND GARDEN,0


In [17]:
c = train.groupby(["family", "store_nbr"]).tail(60).groupby(["family", "store_nbr"]).sales.sum().reset_index()

,family,store_nbr,sales
54,BABY CARE,2,0.00
62,BABY CARE,10,0.00
64,BABY CARE,12,0.00
65,BABY CARE,14,0.00
76,BABY CARE,26,0.00
82,BABY CARE,32,0.00
83,BABY CARE,33,0.00
87,BABY CARE,37,0.00
88,BABY CARE,38,0.00
92,BABY CARE,42,0.00


How to catch the trends, seasonality and anomalies for families

In [ ]:
# 트레인 데이터와 가게 데이터 Merge
d = pd.merge(train, stores)
d["store_nbr"] = d["store_nbr"].astype("int8")
d["year"] = d.date.dt.year

In [21]:
# 시계열과 관련된 특성추가 + 휴일 데이터 병합
def create_date_features(df):
    df['month'] = df.date.dt.month.astype("int8")
    df['day_of_month'] = df.date.dt.day.astype("int8")
    df['day_of_year'] = df.date.dt.dayofyear.astype("int16")
    df['week_of_month'] = (df.date.apply(lambda d: (d.day-1) // 7 + 1)).astype("int8")
    df['week_of_year'] = (df.date.dt.weekofyear).astype("int8")
    df['day_of_week'] = (df.date.dt.dayofweek + 1).astype("int8")
    df['year'] = df.date.dt.year.astype("int32")
    df["is_wknd"] = (df.date.dt.weekday // 4).astype("int8")
    df["quarter"] = df.date.dt.quarter.astype("int8")
    df['is_month_start'] = df.date.dt.is_month_start.astype("int8")
    df['is_month_end'] = df.date.dt.is_month_end.astype("int8")
    df['is_quarter_start'] = df.date.dt.is_quarter_start.astype("int8")
    df['is_quarter_end'] = df.date.dt.is_quarter_end.astype("int8")
    df['is_year_start'] = df.date.dt.is_year_start.astype("int8")
    df['is_year_end'] = df.date.dt.is_year_end.astype("int8")
    # 0: Winter - 1: Spring - 2: Summer - 3: Fall
    df["season"] = np.where(df.month.isin([12,1,2]), 0, 1)
    df["season"] = np.where(df.month.isin([6,7,8]), 2, df["season"])
    df["season"] = pd.Series(np.where(df.month.isin([9, 10, 11]), 3, df["season"])).astype("int8")
    return df
d = create_date_features(d)


holidays["date"] = pd.to_datetime(holidays.date)
holidays_national = holidays[holidays['locale']=='National']
holidays_local = holidays[holidays['locale']=='Local']
holidays_regional = holidays[holidays['locale']=='Regional']

holidays_national[holidays_national.type=='Work Day']
holidays_national.rename(columns = {'type' : 'national_type'}, inplace=True)
holidays_national.rename(columns = {'description' : 'holiday_national'}, inplace=True)
holidays_national.drop(columns = ['locale', 'locale_name', 'transferred'], inplace=True)

holidays_local.rename(columns = {'type' : 'local_type'}, inplace=True)
holidays_local.rename(columns = {'description' : 'holiday_local'}, inplace=True)
holidays_local.rename(columns = {'locale_name' : 'city'}, inplace=True)
holidays_local.drop(columns = ['locale', 'transferred'], inplace=True)

holidays_regional.rename(columns = {'type' : 'regional_type'}, inplace=True)
holidays_regional.rename(columns = {'description' : 'holiday_regional'}, inplace=True)
holidays_regional.rename(columns = {'locale_name' : 'state'}, inplace=True)
holidays_regional.drop(columns = ['locale', 'transferred'], inplace=True)

d = d.merge(holidays_national, how='left', on='date')
d = d.merge(holidays_local, how='left', on=['date', 'city'])
d = d.merge(holidays_regional, how='left', on=['date', 'state'])

d = d.fillna('None')

d['holiday_binary'] = ((d['holiday_national']!='None')|(d['holiday_local']!='None')|(d['holiday_regional']!='None')).astype(int)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_33236\411136244.py:6: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.

C:\Users\Administrator\AppData\Local\Temp\ipykernel_33236\411136244.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Administrator\AppData\Local\Temp\ipykernel_33236\411136244.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Administrator\AppData\Local\Temp\ipykernel_33236\411136244.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveat

In [22]:
# 맨처음값(row=0) fill
oil = oil.fillna(method='bfill').drop(columns='dcoilwtico')

In [23]:
# oil 데이터 Merge
df = d.merge(oil, how='left', on = 'date')
df.head(15)

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,year,month,day_of_month,day_of_year,week_of_month,week_of_year,day_of_week,is_wknd,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,season,national_type,holiday_national,local_type,holiday_local,regional_type,holiday_regional,holiday_binary,dcoilwtico_interpolated
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0.00,Quito,Pichincha,D,13,2013,1,1,1,1,1,2,0,1,1,0,1,0,1,0,0,Holiday,Primer dia del ano,None,None,None,None,1,93.14
1,1782,2013-01-02,1,AUTOMOTIVE,2.00,0.00,Quito,Pichincha,D,13,2013,1,2,2,1,1,3,0,1,0,0,0,0,0,0,0,None,None,None,None,None,None,0,93.14
2,3564,2013-01-03,1,AUTOMOTIVE,3.00,0.00,Quito,Pichincha,D,13,2013,1,3,3,1,1,4,0,1,0,0,0,0,0,0,0,None,None,None,None,None,None,0,92.97
3,5346,2013-01-04,1,AUTOMOTIVE,3.00,0.00,Quito,Pichincha,D,13,2013,1,4,4,1,1,5,1,1,0,0,0,0,0,0,0,None,None,None,None,None,None,0,93.12
4,7128,2013-01-05,1,AUTOMOTIVE,5.00,0.00,Quito,Pichincha,D,13,2013,1,5,5,1,1,6,1,1,0,0,0,0,0,0,0,Work Day,Recupero puente Navidad,None,None,None,None,1,93.15
5,8910,2013-01-06,1,AUTOMOTIVE,2.00,0.00,Quito,Pichincha,D,13,2013,1,6,6,1,1,7,1,1,0,0,0,0,0,0,0,None,None,None,None,None,None,0,93.17
6,10692,2013-01-07,1,AUTOMOTIVE,0.00,0.00,Quito,Pichincha,D,13,2013,1,7,7,1,2,1,0,1,0,0,0,0,0,0,0,None,None,None,None,None,None,0,93.20
7,12474,2013-01-08,1,AUTOMOTIVE,2.00,0.00,Quito,Pichincha,D,13,2013,1,8,8,2,2,2,0,1,0,0,0,0,0,0,0,None,None,None,None,None,None,0,93.21
8,14256,2013-01-09,1,AUTOMOTIVE,2.00,0.00,Quito,Pichincha,D,13,2013,1,9,9,2,2,3,0,1,0,0,0,0,0,0,0,None,None,None,None,None,None,0,93.08
9,16038,2013-01-10,1,AUTOMOTIVE,2.00,0.00,Quito,Pichincha,D,13,2013,1,10,10,2,2,4,0,1,0,0,0,0,0,0,0,None,None,None,None,None,None,0,93.81


In [25]:
# Weather API 추가하려 했으나 데이터 누수라고 판단하여 제거
"""
weather = pd.read_csv('./data/weather.csv').iloc[:, 1:]
weather.conditions.value_counts()
weather_condition = weather[['datetimeStr', 'city', 'temp', 'wspd', 'precip', 'conditions', 'humidity']]
weather_condition.replace({'conditions':{'Rain, Partially cloudy':'Rain', 'Rain, Overcast':'Rain', 'Partially cloudy':'Cloudy', 'Overcast':'Cloudy'}}, inplace=True)
weather_condition.drop(index=weather_condition[weather_condition.datetimeStr.isna()].index , inplace=True)
weather_condition.drop(index=weather_condition[weather_condition.temp.isna()].index , inplace=True)
weather_condition['date'] = weather_condition.datetimeStr.str.split('T').str[0].astype('datetime64[ns]')
weather_condition.drop(columns='datetimeStr', inplace=True)
df_final = df.merge(weather_condition, how='left', on=['date', 'city'])

"""

"\nweather = pd.read_csv('./data/weather.csv').iloc[:, 1:]\nweather.conditions.value_counts()\nweather_condition = weather[['datetimeStr', 'city', 'temp', 'wspd', 'precip', 'conditions', 'humidity']]\nweather_condition.replace({'conditions':{'Rain, Partially cloudy':'Rain', 'Rain, Overcast':'Rain', 'Partially cloudy':'Cloudy', 'Overcast':'Cloudy'}}, inplace=True)\nweather_condition.drop(index=weather_condition[weather_condition.datetimeStr.isna()].index , inplace=True)\nweather_condition.drop(index=weather_condition[weather_condition.temp.isna()].index , inplace=True)\nweather_condition['date'] = weather_condition.datetimeStr.str.split('T').str[0].astype('datetime64[ns]')\nweather_condition.drop(columns='datetimeStr', inplace=True)\ndf_final = df.merge(weather_condition, how='left', on=['date', 'city'])\n\n"

In [26]:
# id 특성 제거
df.drop(columns='id', inplace=True)

In [27]:
# 이외에도 존재할 수 있는 Missing Value를 위한 함수
columns_missing = df.columns[df.isnull().any()].tolist()

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
columns_missing_numeric = df[columns_missing].select_dtypes(include=numerics).columns.tolist()
columns_missing_object = list(set(columns_missing) - set(columns_missing_numeric))

In [28]:
# Numeric 변수는 Interpolation으로 채워넣기
for i in columns_missing_numeric:
    interpolation = df[['date', i]]
    df[i] =interpolation[i].interpolate()

In [29]:
# Object 변수는 앞의 row 값으로 채워넣기
for i in columns_missing_object:
    df[i] = df[i].fillna(method='ffill')

In [30]:
# 데이터가 너무 커서 랜덤으로 도시 5개로 한하여 분석 데이터 설정
import numpy as np
np.random.seed(2)
city_rand = np.random.choice(df.city.unique(), 5, replace=False)
df = df[df['city'].isin(city_rand)]

In [31]:
# 데이터가 너무 커서 제품군 매출 상위 9개로 분석 데이터 설정
top_product = df.groupby('family')[['sales']].sum().sort_values(by='sales', ascending=False).iloc[0:9].index
df = df[df['family'].isin(top_product)]

In [32]:
# Clean Data 내보내기
df.to_csv('./data/clean_data.csv', index=0)